<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preprocessing-the-datasets" data-toc-modified-id="Preprocessing-the-datasets-1">Preprocessing the datasets</a></span><ul class="toc-item"><li><span><a href="#To-do" data-toc-modified-id="To-do-1.1">To do</a></span><ul class="toc-item"><li><span><a href="#Stopwords" data-toc-modified-id="Stopwords-1.1.1">Stopwords</a></span></li><li><span><a href="#Punctuation" data-toc-modified-id="Punctuation-1.1.2">Punctuation</a></span></li><li><span><a href="#Lematizer" data-toc-modified-id="Lematizer-1.1.3">Lematizer</a></span></li><li><span><a href="#The-3-commented-cells-below-compute-the-lemmatized-version-of-the-df--&gt;-takes-a-long-time-to-run" data-toc-modified-id="The-3-commented-cells-below-compute-the-lemmatized-version-of-the-df-->-takes-a-long-time-to-run-1.1.4">The 3 commented cells below compute the lemmatized version of the df -&gt; takes a long time to run</a></span></li></ul></li><li><span><a href="#The-part-where-we-compute-the-predictions-of-dummy-classifier-trained-on-the-preprocessed-datasets" data-toc-modified-id="The-part-where-we-compute-the-predictions-of-dummy-classifier-trained-on-the-preprocessed-datasets-1.2">The part where we compute the predictions of dummy classifier trained on the preprocessed datasets</a></span></li></ul></li></ul></div>

# Preprocessing the datasets

good read : https://www.analyticsvidhya.com/blog/2018/07/hands-on-sentiment-analysis-dataset-python/

In [2]:
import pandas as pd
from utils import open_by_tweets
from utils import *
import nltk
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt

#if wordcloud not already installed, run the following command : !python -m pip install wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

#uncomment the comments below if the packages have not already been downloaded

#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')


In [2]:
#load tweets in list where each element is a tweet

with open('data/train_pos.txt',"r") as file:
    train_pos = file.read().split('\n')
#train_pos.remove('')

train_pos = pd.DataFrame({'tweet' : train_pos}) #create the dataframe
 
#train_pos = pd.read_csv('data/train_pos.txt', sep="\n", header=None)    
    
with open('data/train_neg.txt',"r") as file:
    train_neg = file.read().split('\n')
#train_neg.remove('')
train_neg = pd.DataFrame({'tweet' : train_neg})
    
#let's check it worked correctly
train_pos


,tweet
0,<user> i dunno justin read my mention or not ....
1,"because your logic is so dumb , i won't even c..."
2,""" <user> just put casper in a box ! "" looved t..."
3,<user> <user> thanks sir > > don't trip lil ma...
4,visiting my brother tmr is the bestest birthda...
5,<user> yay ! ! #lifecompleted . tweet / facebo...
6,<user> #1dnextalbumtitle : feel for you / roll...
7,workin hard or hardly workin rt <user> at hard...
8,<user> i saw . i'll be replying in a bit .
9,this is were i belong


## To do 
**Implement everything in methods, we need a pipeline to process test data using the same steps as for the train data**
* Remove blank tweets: Zeineb
* Open contractions & transform @ into at or u into you, etc...: Zeineb
* Separate punctuation from words like : 'word...' becomes 'word ...'
* Separate alphabet & numbers (e.g. 398stomach)
* removing repetitive letters inside word, ie, 'loooove' (seems complicated to do : https://stackoverflow.com/questions/20516100/term-split-by-hashtag-of-multiple-words but maybe we could try https://github.com/matchado/HashTagSplitter)
* lemmatization and/or stemming (https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8): DONE
* removing irrelevant punctuation (keep ! and ?, watch out for emojis, maybe remove isolated punctuation (there's a space before and after)) & when there are two exclamation points (for ex) we can transform them into a word (like doubleinterrogation): Lilia
* stopwords, keep relevant ones: Lilia
* transforming all variants of "haha" to "haha" : Zeineb

### Stopwords

In [3]:
#list of stopword from nltk + "url" added -> maybe we should remove some of them, ie., 'but'
stopWords = set(stopwords.words('english'))
stopWords.add("url")
stopWords.add("user")
print(stopWords)


{'him', 'as', 'for', 'when', 'this', 'couldn', 'under', "weren't", 't', 'should', 've', 'they', "mightn't", "shan't", 'both', 'further', "aren't", "doesn't", 'once', 'do', 'theirs', 'wasn', 'wouldn', 'user', 'are', 'few', 'ain', "mustn't", 'you', "don't", "you've", 'hasn', "needn't", "shouldn't", 'these', 'now', 'yourselves', 'had', 'by', 'has', 'or', 'o', 'did', 'more', 'again', 'been', 'at', 'where', 'nor', 'which', 'from', 'so', "won't", "wasn't", 'here', 'y', 'an', 'ourselves', 'it', 'between', 'in', 'being', 'while', 'those', 'whom', 'our', 'during', 'up', 'but', 'll', 'i', 'how', 'on', 'why', 'through', 'having', 'very', 'about', 'same', 'below', 'shouldn', "hasn't", 'other', 'url', 'because', 'with', 'over', "isn't", 'his', 'themselves', 'them', 'aren', 'her', 'a', 'don', 'won', 'herself', "you're", 'out', 'against', 'its', 'then', 'ours', 'above', "hadn't", 'after', 'there', 'hers', 'before', 'their', 'he', 'm', 'does', 'that', 'is', 'of', 'd', "wouldn't", 'needn', 'doing', 'al

Studying the occurrence of stopwords in the datasets

In [ ]:
# alternate way 
# Create an empty dictionary 
d_pos = dict() 
# Loop through each line of the file 
for tweet in train_pos: 
    # Remove the leading spaces and newline character 
    tweet = tweet.strip() 
  
    # Convert the characters in line to  
    # lowercase to avoid case mismatch 
    tweet = tweet.lower() 
  
    # Split the line into words 
    words = tweet.split(" ") 
  
    # Iterate over each word in line 
    for word in words: 
        # Check if the word is already in dictionary 
        if word in d_pos: 
            # Increment count of word by 1 
            d_pos[word] = d_pos[word] + 1
        else: 
            # Add the word to dictionary with count 1 
            d_pos[word] = 1
            
#Print contents of dictionary
for key in list(d_pos.keys()): 
    print(key, ":", d_pos[key]) 

In [ ]:
# Create an empty dictionary 
d_neg = dict() 
# Loop through each line of the file 
for tweet in train_neg: 
    # Remove the leading spaces and newline character 
    tweet = tweet.strip() 
  
    # Convert the characters in line to  
    # lowercase to avoid case mismatch 
    tweet = tweet.lower() 
  
    # Split the line into words 
    words = tweet.split(" ") 
  
    # Iterate over each word in line 
    for word in words: 
        # Check if the word is already in dictionary 
        if word in d_neg: 
            # Increment count of word by 1 
            d_neg[word] = d_neg[word] + 1
        else: 
            # Add the word to dictionary with count 1 
            d_neg[word] = 1
            
#Print contents of dictionary
for key in list(d_neg.keys()):
    print(key, ":", d_neg[key]) 

In [ ]:
# for stopword in stopWords:
#     plt.figure(figsize=(3, 2))
#     plt.title("Histogram for {}".format(stopword))
#     ax = sns.barplot(['pos', 'neg'],[count_vect_pos.vocabulary_.get(stopword), count_vect_neg.vocabulary_.get(stopword)], palette=['green', 'red'])

for stopword in stopWords:
    plt.figure(figsize=(3, 2))
    plt.title("Histogram for {}".format(stopword))
    ax = sns.barplot(['pos', 'neg'],[d_pos.get(stopword), d_neg.get(stopword)], palette=['green', 'red'])

### Punctuation

In [ ]:
print(string.punctuation)

In [ ]:
for tweet in train_neg:
    if  ('-' in tweet):
        print(tweet)

print(count)

In [ ]:
for punct in string.punctuation:
    plt.figure(figsize=(3, 2))
    plt.title("Histogram for {}".format(punct))
    ax = sns.barplot(['pos', 'neg'],[d_pos.get(punct), d_neg.get(punct)], palette=['green', 'red'])

In [ ]:
# keep this punctuation
len('!#&()-/:<>[]^|}')

### Lematizer

In [4]:
#for the lemmatizer to really work, we need to give a tag to indicate the nature 
#of the word (adv, adj, verb,noun,...) because if we don't, it treats every word
#as a noun (ie, went stays went instead of go)

#taken from : https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    #pos_tag doesn't have the same tag than wordnet therefore we have to modify the tag
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def lemmatize_sentence(sentence):
    """
    lemmatize a given sentence by first giving a tag for each word and then lemmatizing
    the word based on its tag
     
    Returns:
    list:contains the lemmatized words of the parameter sentence
    """
    lemmatizer = nltk.wordnet.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]

Remarks : 
- I notice that 'ing'-form of verbs are often 'in' and are therefore not classify as verb during lemmatization

In [ ]:
# AVOID USING IT (WILL PROBABLY BE REMOVED)
def preprocess_tweet(tweets, punctuation = string.punctuation, stopwords = stopWords):
    """
    preprocessed the given list of tweets :
        - lowercase
        - remove punctation
        - remove stopwords
        - lemmatize
         
        
    Returns:
    list of the tokenized and preprocessed version of the tweets
    """
    
    #convert every tweet to lowercase
    tweets = [tweet.lower() for tweet in tweets]
    
    #remove punctuation
    tweets = [tweet.translate(str.maketrans('', '', punctuation)) for tweet in tweets]
    
    tweets_mod = []
    for tweet in tweets:
        #lemmatize the sentence and give back a list of words
        words = lemmatize_sentence(tweet) 
        
        #remove the words that belong to stopwords (list)
        words = [word for word in words if (word not in stopwords)]
        
        tweets_mod.append(words)
        
    return tweets_mod

In [5]:

def preprocess_df(df, punctuation = string.punctuation, stopwords = stopWords, lemmatize = False):
    """
    preprocessed the given list of tweets :
        - lowercase
        - remove punctation
        - remove stopwords
        - lemmatize the words if lemmatize argument set to True
         
        
    Returns:
    initial dataframe preprocessed
    """
    
    df['tweet'] = df['tweet'].apply(lambda x : x.lower()) #lowercase
    df['tweet'] = df['tweet'].apply(lambda x : x.translate(str.maketrans('', '', punctuation))) #remove punctations
    if(lemmatize):
        df['tweet'] = df['tweet'].apply(lambda x : lemmatize_sentence(x))
    else:
        df['tweet'] = df['tweet'].apply(lambda x : x.split())
        
    df['tweet'] = df['tweet'].apply(lambda x: [word for word in x if (word not in stopWords)])#remove stopwords
    df['tweet'] = df['tweet'].apply(lambda x : ' '.join(x))
    
    return df

In [6]:
train_pos_clean = preprocess_df(train_pos)
train_neg_clean = preprocess_df(train_neg)

### The 3 commented cells below compute the lemmatized version of the df -> takes a long time to run

In [ ]:
#train_pos_lem = preprocess_df(train_pos, lemmatize=True)

In [ ]:
#train_neg_lem = preprocess_df(train_neg, lemmatize=True)

In [ ]:
#train_pos_lem.to_csv("train_pos_lem.csv")
#train_neg_lem.to_csv("train_neg_lem.csv")

In [ ]:
#example
#preprocess_tweet(["hEllo I went to the movies and ate chips!!!!!", "#12"])

In [ ]:
def cloud_words(tokenized_tweets):
    """
    Plot a wordcloud of every word in all tweets
    """
    #we need a string with all the words of all the tweets
    all_words = [item for sublist in tokenized_tweets for item in sublist]#list of (list of words) -> list of (sentence)
    all_words = ' '.join(word for word in all_words)

    wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

    plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()


In [ ]:
#cloud_words(train_pos_tok)

In [ ]:
#cloud_words(train_neg_tok)

## The part where we compute the predictions of dummy classifier trained on the preprocessed datasets

In [7]:
df_cleaned = merge_datasets_with_labels(train_pos_clean, train_neg_clean)
#df_cleaned_lem = merge_datasets_with_labels(train_pos_lem, train_neg_lem)

In [8]:
clf, count_vect = create_tfidf_clf(df_cleaned,ngram=(2,3)) #accuracy of preprocessed model without lemmatization
#clf_lem, count_vect_lem = create_tfidf_clf(df_cleaned_lem,ngram=(2,3))

Our model's accuracy is 0.748345033099338


In [9]:
clf_1, count_vect_1 = create_tfidf_clf(df_cleaned,ngram=(1,1)) #accuracy of preprocessed model without lemmatization
#clf_lem_1, count_vect_lem_1 = create_tfidf_clf(df_cleaned_lem,ngram=(1,1))

Our model's accuracy is 0.7667446651066979


In [10]:
df_unknown = test_dataset_to_df('data/test_data.txt')

In [ ]:
df_unknown

In [11]:
df_unknown_clean = preprocess_df(df_unknown)#preprocessed the dataframe to predict
df_unknown_clean

,tweet
1,sea doo pro sea scooter sports portable seadoo...
2,shucks well work week cant come cheer oh put b...
3,cant stay away bug thats baby
4,maam lol im perfectly fine contagious anymore ...
5,whenever fall asleep watching tv always wake h...
6,needs get rid thing scares lol dont need car e...
7,whatever terrible mood
8,yesss rt thanks jordan love im gonna call later
9,friend text check last night
10,followback please ur unitytour come europe sweden


In [12]:
predict_tfidf_clf(df_unknown_clean,clf_1,count_vect_1, "dummy_without_lem1.csv")

In [ ]:
#df_unknown_clean_lem = preprocess_df(df_unknown,lemmatize=True)
#predict_tfidf_clf(df_unknown_clean_lem,clf_lem,count_vect_lem, "dummy_with_lem.csv")